In [2]:
# CONSTANTS
bid=1500
approve=0.25
cr=0.01
ctr=0.015
epc=cr*approve*bid
print(epc)
ecpm=epc*ctr*1000
print(ecpm)
campaigns = None

3.75
56.24999999999999


In [3]:
import pandas as pd
import clickhouse_driver as ch
from dotenv import load_dotenv
import os
from scipy.optimize import minimize
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score

from hw import HoltWinters
from cross_val import CVScore

with open('./queries/q2.sql', 'r') as f:
    q = f.read()
q=q.replace('${BID}', str(bid))
sqlCH = ch.Client(host="192.168.235.72",
                    user="valferov",
                    password="bUfMupl1kDgK1337")

df = pd.DataFrame(sqlCH.execute(q))

In [4]:
df.columns = ['campaigns', 'adv_id', 'shows', 
                'postbacks', 'ecpm']


In [5]:
df['diff'] = abs(df['ecpm']-ecpm)
df['geo'] = df['campaigns'].apply(lambda x: x[:2])

In [9]:
campaignsPriority = ['RU', 'KZ', 'UA', 'BY',
                        'UZ', 'DE', 'HU', 'HR',
                        'LT', 'LV', 'SK', 'RO',
                        'PL', 'ES']
df_diff = df.loc[df['diff'] == df['diff'].min()]
for i in campaignsPriority:
    check = df_diff.loc[df_diff['geo'] == i]
    if not check.empty:
        campaigns = check['campaigns'].values[0+1]
        break
if campaigns is None:
    print('sas')
    campaigns = df_diff.sort_values(by='shows', 
                                    ascending=False)['campaigns'].values[0]
campaigns

'DE | папилломы'

In [54]:
df_diff['geo'].drop_duplicates()

2      HU
3      DE
17     BG
19     CZ
21     GR
32     HR
34     PT
41     SK
48     LT
62     LV
67     RS
80     RO
97     AT
108    PL
162    ES
163    SI
181    IT
456    те
673    Te
Name: geo, dtype: object

In [ ]:
campaignId = 12862

with open('./queries/q2.sql', 'r') as f:
    q = f.read()
q=q.replace('${CAMPAIGNS_ID}', str(campaignId))
sqlCH = ch.Client(host="192.168.235.72",
                    user="valferov",
                    password="bUfMupl1kDgK1337")

df = pd.DataFrame(sqlCH.execute(q))